# Word Embedding using python and Keras

In [2]:
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wikipedia

Loading sample text from Wikipedia

In [4]:
# Read the file wiki_pages.txt in a list with a line per element
with open('wiki_pages.txt') as f:
    wiki_pages = f.readlines()

for i in range(len(wiki_pages)):
    wiki_pages[i] = wiki_pages[i].replace('\n', '')

In [79]:
def clean_text(text):
    chars_to_remove = ['\n', '\t', '\r', ',', '.', '!']
    for c in chars_to_remove:
        text = text.replace(c, ' ')
    return text

wikipedia.set_lang("it")
train_text = []	
for page in wiki_pages:
    all_articles = wikipedia.search(page)
    
    for article in all_articles:
        try:
            article = wikipedia.page(article)
            train_text.append(clean_text(article.content))
            print(article.title)
        except:
            print("Page not found")

Italia
Italia 1
Nazionale di calcio dell'Italia
Italia settentrionale
Viaggio in Italia (film)
Rai
Regno d'Italia (1861-1946)
Anniversario della liberazione d'Italia
Italia meridionale
ITA Airways
Unione europea
Stati membri dell'Unione europea
Allargamento dell'Unione europea
Targhe d'immatricolazione nell'Unione europea
Comunità economica europea
Consiglio dell'Unione europea
Istituzioni dell'Unione europea
Commissione europea
Adesione della Turchia all'Unione europea
Uscita del Regno Unito dall'Unione europea
Diritti umani
Dichiarazione universale dei diritti umani
Corte europea dei diritti dell'uomo
Dichiarazione dei diritti dell'uomo e del cittadino
Corea del Nord
Diritti umani in India
Diritti umani negli Stati Uniti d'America
Consiglio per i diritti umani delle Nazioni Unite
Tre generazioni dei diritti umani
Diritti umani in Bangladesh
Bruxelles
Bruxelles (comune)
Regione di Bruxelles-Capitale
Grand-Place (Bruxelles)
Belgio
Dio esiste e vive a Bruxelles
Aeroporto di Bruxelles-Na

In [80]:
# Create a list of unique words in the text
words = []
for text in train_text:
    words += text.split()
unique_words = list(set(words))
unique_words_count = len(unique_words)
print("Unique words =", unique_words_count)

Unique words = 27509


In [81]:
unique_words_index = {unique_words[i]:i for i in range(unique_words_count)}

## Creation of data to fit

The input of the neural network is a vector of $1$ (only in the position of the current word) and $0$ (in all the other positions). The output of the network should be the next word in the text (encoded as the same type of vector with $0$ and $1$)

In [84]:
x = np.zeros((len(words), unique_words_count), dtype=bool)
y = np.zeros((len(words), unique_words_count), dtype=bool)
for i in range(len(words)-1):
    x[i, unique_words_index[words[i]]] = 1
    y[i, unique_words_index[words[i+1]]] = 1
    
# Split into train and test data
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

## Create deep neural network

Each input neuron correspond to 1 word and the number of neuron in the first hidden layer corresponds to the number of dimension of the word embedding. After that we have another fully connected layer that pass through a softmax to output the next word probability. This network is trained to predict the next word given a single word in a sentence.

In [85]:
from keras.layers import Input, Dense, Softmax
from keras.models import Model

def create_network(input_neurons, word_embedding_dimension):
    xin = Input(shape=(input_neurons,))
    x = Dense(word_embedding_dimension, activation='relu')(xin)
    x = Dense(input_neurons, activation='softmax')(x)
    
    return Model(inputs=xin,outputs=x)

WORD_EMBEDDING_DIMENSION = 2

model = create_network(unique_words_count, WORD_EMBEDDING_DIMENSION)
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 27509)]           0         
                                                                 
 dense_11 (Dense)            (None, 2)                 55020     
                                                                 
 dense_12 (Dense)            (None, 27509)             82527     
                                                                 
Total params: 137547 (537.29 KB)
Trainable params: 137547 (537.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [86]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [87]:
model.fit(x_train,y_train, shuffle=True, epochs=100, batch_size=32,validation_data=(x_test,y_test))

Epoch 1/100
4309/4309 [==============================] - 39s 9ms/step - loss: 8.4181 - accuracy: 0.0365 - val_loss: 8.1517 - val_accuracy: 0.0363
Epoch 2/100
4309/4309 [==============================] - 33s 8ms/step - loss: 7.8347 - accuracy: 0.0367 - val_loss: 8.2939 - val_accuracy: 0.0363
Epoch 3/100
4309/4309 [==============================] - 35s 8ms/step - loss: 7.7604 - accuracy: 0.0367 - val_loss: 8.4455 - val_accuracy: 0.0362
Epoch 4/100
4309/4309 [==============================] - 37s 9ms/step - loss: 7.7172 - accuracy: 0.0366 - val_loss: 8.5282 - val_accuracy: 0.0362
Epoch 5/100
4309/4309 [==============================] - 37s 8ms/step - loss: 7.6755 - accuracy: 0.0367 - val_loss: 8.5528 - val_accuracy: 0.0370
Epoch 6/100
4309/4309 [==============================] - 35s 8ms/step - loss: 7.6207 - accuracy: 0.0363 - val_loss: 8.5514 - val_accuracy: 0.0368
Epoch 7/100
4309/4309 [==============================] - 36s 8ms/step - loss: 7.5565 - accuracy: 0.0363 - val_loss: 8.5352 -

In [88]:
# Save the model
model.save('word_embedding_model.h5')

c:\Users\Luca\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [65]:
unique_words[np.argmax(x_test[0:1])]

'in'

In [66]:
def predict_word(model, input_word):
    input_vector = np.zeros((1, unique_words_count))
    input_vector[0, unique_words_index[input_word]] = 1
    return unique_words[np.argmax(model.predict(input_vector))]

In [68]:
predict_word(model, 'di')

1/1 [==============================] - 0s 24ms/step


'e'

### Extracting the word embedding from the neural network

In [89]:
# The embedding of each word is defined by the weights of the first layer
word_embedding = model.layers[1].get_weights()[0]
word_embedding.shape

(27509, 2)

In [90]:
word_embedding[unique_words_index['di']]

array([-0.70760226, -0.5365534 ], dtype=float32)

In [91]:
word_embedding[unique_words_index['e']]

array([-0.23127148, -0.02789657], dtype=float32)

In [92]:
def find_closest_words(words_embeddings, word, n=5):
    word_index = unique_words_index[word]
    word_embedding = words_embeddings[word_index]
    distances = np.linalg.norm(words_embeddings - word_embedding, axis=1)
    closest_words = [unique_words[i] for i in np.argsort(distances)[1:n+1]]
    return closest_words

In [97]:
find_closest_words(word_embedding, 'Roma')

['narrativa', 'rivelano', 'contrario', 'RaiPlay', 'decenni']

In [94]:
test = word_embedding[unique_words_index['Italia']] - word_embedding[unique_words_index['Roma']] + word_embedding[unique_words_index['Parigi']]
distances = np.linalg.norm(word_embedding - test, axis=1)
closest_words = [unique_words[i] for i in np.argsort(distances)[0:5]]
closest_words

['2000', 'documento', 'Rice', 'ricorrenza', 'Clarisse']